- pm_train.csv와 pm_test.csv는 항공기 엔진에 대한 예측정비와 관련된 데이터입니다.
- 두 데이터를 읽으신 후, ttf 컬럼이 30일 이하인 경우, 예측 대상이라 고려하여 1을, 그렇지 않은 경우 0 값을 갖는 label 컬럼을 새로 만들어서 추가하세요

- setting1-setting3, 그리고 s1-s21 변수를 X 변수로 하고, label 변수를 target으로 하여, 인공신경망으로 피팅해보세요.

- 피팅된 모형에 test셋을 넣어 예측하고, 예측 결과는 sklearn의 classification_report를 통해 해석해보세요.

In [1]:
import pandas as pd

df_train = pd.read_csv("pm_train.csv")
df_test = pd.read_csv("pm_test.csv")

period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()


,Unnamed: 0,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,...,s14,s15,s16,s17,s18,s19,s20,s21,ttf,label_bc
0,0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0
1,1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0
2,2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0
3,3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0
4,4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0


데이터 전처리

In [2]:
df_train.isnull().mean()

Unnamed: 0    0.0
id            0.0
cycle         0.0
setting1      0.0
setting2      0.0
setting3      0.0
s1            0.0
s2            0.0
s3            0.0
s4            0.0
s5            0.0
s6            0.0
s7            0.0
s8            0.0
s9            0.0
s10           0.0
s11           0.0
s12           0.0
s13           0.0
s14           0.0
s15           0.0
s16           0.0
s17           0.0
s18           0.0
s19           0.0
s20           0.0
s21           0.0
ttf           0.0
label_bc      0.0
dtype: float64

In [3]:
#Column 선택
df_train.drop( ['Unnamed: 0', 'id', 'ttf'], axis=1, inplace=True)
df_test.drop( ['Unnamed: 0', 'id', 'ttf'], axis=1, inplace=True)
df_train.head()

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,label_bc
0,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,0
1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,0
2,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,0
3,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,0
4,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,0


In [26]:
#X, Y
X_train = df_train.drop("label_bc", axis=1)
y_train = df_train.label_bc
X_test = df_test.drop("label_bc", axis=1)
y_test = df_test.label_bc

#scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame( scaler.fit_transform(X_train), columns = X_train.columns )
X_test_scaled = pd.DataFrame( scaler.fit_transform(X_test), columns = X_test.columns )


In [30]:
#SMOTE for upsampling smaller cases
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 1 ,k_neighbors = 5, random_state=1)   
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train.ravel())

# Ensemble - Bagging

In [11]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

#defining a set of values as a dictionary for hyperparameters
param_grid = {
    "n_estimators":[200,300]  
}

#declaring GridSearchCV model
model = GridSearchCV(
    estimator = clf,
    param_grid = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5    
)

#fitting values to the gridsearchcv model
model.fit(X_train_res, y_train_res)

#printing the best possible values to enhance accuracy
print(model.best_params_)
print(model.best_estimator_)

#printing the best score
print(model.best_score_)



Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5; 1/2] START n_estimators=200............................................
[CV 1/5; 1/2] END .............n_estimators=200;, score=0.967 total time=  15.7s
[CV 2/5; 1/2] START n_estimators=200............................................
[CV 2/5; 1/2] END .............n_estimators=200;, score=0.973 total time=  12.9s
[CV 3/5; 1/2] START n_estimators=200............................................
[CV 3/5; 1/2] END .............n_estimators=200;, score=0.979 total time=  12.8s
[CV 4/5; 1/2] START n_estimators=200............................................
[CV 4/5; 1/2] END .............n_estimators=200;, score=0.962 total time=  12.7s
[CV 5/5; 1/2] START n_estimators=200............................................
[CV 5/5; 1/2] END .............n_estimators=200;, score=0.974 total time=  12.7s
[CV 1/5; 2/2] START n_estimators=300............................................
[CV 1/5; 2/2] END .............n_estimators=300;,

In [12]:
#분류 보고서
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
results = model.predict( X_test)
print('\n', classification_report(y_test, results))


               precision    recall  f1-score   support

           0       0.99      0.99      0.99     12764
           1       0.66      0.76      0.71       332

    accuracy                           0.98     13096
   macro avg       0.83      0.87      0.85     13096
weighted avg       0.99      0.98      0.98     13096



Ensemble - Boosting

In [13]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
    "n_estimators":[100,200],
    "max_depth":[3,5],
    "reg_lambda":[.01,.1]    
}

model = GridSearchCV(
    estimator = clf,
    param_grid = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5    
)

#fitting values to the gridsearchcv model
model.fit(X_train_res, y_train_res)

#printing the best possible values to enhance accuracy
print(model.best_params_)
print(model.best_estimator_)

#printing the best score
print(model.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START max_depth=3, n_estimators=100, reg_lambda=0.01..............
[CV 1/5; 1/8] END max_depth=3, n_estimators=100, reg_lambda=0.01;, score=0.946 total time=   2.5s
[CV 2/5; 1/8] START max_depth=3, n_estimators=100, reg_lambda=0.01..............
[CV 2/5; 1/8] END max_depth=3, n_estimators=100, reg_lambda=0.01;, score=0.962 total time=   2.4s
[CV 3/5; 1/8] START max_depth=3, n_estimators=100, reg_lambda=0.01..............
[CV 3/5; 1/8] END max_depth=3, n_estimators=100, reg_lambda=0.01;, score=0.972 total time=   2.4s
[CV 4/5; 1/8] START max_depth=3, n_estimators=100, reg_lambda=0.01..............
[CV 4/5; 1/8] END max_depth=3, n_estimators=100, reg_lambda=0.01;, score=0.955 total time=   2.4s
[CV 5/5; 1/8] START max_depth=3, n_estimators=100, reg_lambda=0.01..............
[CV 5/5; 1/8] END max_depth=3, n_estimators=100, reg_lambda=0.01;, score=0.967 total time=   2.4s
[CV 1/5; 2/8] START max_depth=3, n_estimators

In [14]:
#분류 보고서
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
results = model.predict( X_test)
print('\n', classification_report(y_test, results))


               precision    recall  f1-score   support

           0       0.99      0.99      0.99     12764
           1       0.67      0.77      0.72       332

    accuracy                           0.98     13096
   macro avg       0.83      0.88      0.85     13096
weighted avg       0.99      0.98      0.99     13096



In [16]:
!pip install optuna
import optuna
import sklearn
#import XGboost
from xgboost import XGBClassifier

def objective(trial):
    reg_lambda = trial.suggest_uniform('reg_lambda',0.2,0.3)
    n_estimators = trial.suggest_int('n_estimators',200,300)
    max_depth = trial.suggest_int('max_depth',3,8)
    max_features = trial.suggest_uniform('max_features',0.01,1)

    clf = XGBClassifier(
      n_estimators= n_estimators,
     reg_lambda=reg_lambda,
     max_depth=max_depth,
     max_features= max_features)    
    return sklearn.model_selection.cross_val_score(clf, X_train_res, y_train_res, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[I 2022-09-22 06:59:14,040] A new study created in memory with name: no-name-f4f09bb5-3d0e-483e-803c-5dec1f546ec2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  # This is added back by InteractiveShellApp.init_path()
[I 2022-09-22 06:59:42,498] Trial 0 finished with value: 0.9657752497109712 and parameters: {'reg_lambda': 0.22789473359744214, 'n_estimators': 245, 'max_depth': 6, 'max_features': 0.0651399438343796}. Best is trial 0 with value: 0.9657752497109712.
/usr/local/lib/p

In [17]:
#분류 보고서
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
results = model.predict( X_test)
print('\n', classification_report(y_test, results))


               precision    recall  f1-score   support

           0       0.99      0.99      0.99     12764
           1       0.67      0.77      0.72       332

    accuracy                           0.98     13096
   macro avg       0.83      0.88      0.85     13096
weighted avg       0.99      0.98      0.99     13096



DNN

In [31]:
import pandas as pd
from tensorflow.keras.utils import to_categorical #pandas의 get_dummies


y_train = to_categorical( y_train_res )
y_test = to_categorical( y_test )

In [37]:
y_train.shape
X_train_res

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,0.000000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662
1,0.002770,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014
2,0.005540,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375
3,0.008310,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386
4,0.011080,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35057,0.496780,0.449411,0.757430,0.0,0.0,0.756714,0.841941,0.737686,0.0,1.0,...,0.259286,0.438554,0.475918,0.800302,0.0,0.674097,0.0,0.0,0.155730,0.227589
35058,0.563585,0.534483,0.648627,0.0,0.0,0.714415,0.719252,0.770574,0.0,1.0,...,0.461943,0.380381,0.762321,0.812592,0.0,0.768040,0.0,0.0,0.234236,0.214271
35059,0.865758,0.435294,0.333333,0.0,0.0,0.664273,0.619795,0.882207,0.0,1.0,...,0.285445,0.587207,0.245677,0.800547,0.0,0.660839,0.0,0.0,0.242858,0.346133
35060,0.385864,0.436213,0.346416,0.0,0.0,0.636735,0.518393,0.772596,0.0,1.0,...,0.213692,0.517015,0.088542,0.503228,0.0,0.570250,0.0,0.0,0.239860,0.244169


In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add( Dense( 12, activation="relu", input_shape=(25,)) )
model.add( Dense( 6, activation="relu") )
model.add( Dense( 2,  activation="softmax"))

model.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

#Gridsearch here


model.fit( X_train_res, y_train, epochs=20)
results = model.predict( X_test)

Epoch 1/20
1096/1096 [==============================] - 2s 1ms/step - loss: 0.2502 - acc: 0.8964
Epoch 2/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1387 - acc: 0.9416
Epoch 3/20
1096/1096 [==============================] - 2s 2ms/step - loss: 0.1325 - acc: 0.9442
Epoch 4/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1293 - acc: 0.9444
Epoch 5/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1273 - acc: 0.9459
Epoch 6/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1260 - acc: 0.9468
Epoch 7/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1248 - acc: 0.9470
Epoch 8/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1235 - acc: 0.9471
Epoch 9/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1229 - acc: 0.9477
Epoch 10/20
1096/1096 [==============================] - 1s 1ms/step - loss: 0.1218 - acc: 0.9479
Epoch 11/20
1096/1096 [======

In [46]:
df_test.label_bc.shape

(13096,)

In [ ]:
#분류 보고서
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

results = model.predict( X_test)
print('\n', classification_report(np.argmax(y_test, axis = -1), np.argmax(results, axis = -1)))